In [60]:
import geojson
from PIL import Image, ImageDraw
from pyproj import Proj, transform
import numpy as np
from simplification.cutil import simplify_coords, simplify_coords_vw, simplify_coords_vwp
import aggdraw
import copy
import timeit
import yaml

DefaultStyle = {'color': 'red', 'width': 2}

In [171]:
def LoadData(fileName):
    with open(fileName, encoding='utf-8') as f:
        gj = geojson.load(f)
        
        for feature in gj['features']:
            
            minP = [999.0, 999.0]
            maxP = [0.0, 0.0]
            centerP = [0.0, 0.0]
            pointsNumber = 0.0
            
            for a in feature['geometry']['coordinates']:
                for b in a:
                    for c in b:
                        c = ToConvertCoordinate(c)
                        
                        #Поиск минимальных и максимальных координат объекта
                        if c[0] < minP[0]:
                            minP[0] = c[0]
                        elif c[0] > maxP[0]:
                            maxP[0] = c[0]
                        if c[1] < minP[1]:
                            minP[1] = c[1]
                        elif c[1] > maxP[1]:
                            maxP[1] = c[1]
                        
                        #Поиск цента масс
                        centerP[0] += c[0]
                        centerP[1] += c[1]
                        pointsNumber += 1
            
            centerP[0] /= pointsNumber
            centerP[1] /= pointsNumber
            
            feature['properties']['minP'] = minP
            feature['properties']['maxP'] = maxP
            feature['properties']['centerP'] = centerP
            feature['properties']['w'] = maxP[0] - minP[0]
            feature['properties']['h'] = maxP[1] - minP[1]
            feature['properties']['density'] = pointsNumber / (maxP[0] - minP[0] + maxP[1] - minP[1]) / 2
        
        return gj['features']

def LoadStyles(fileName="styles.yaml", encoding='utf-8'):
    with open(fileName, 'r') as stream:
        try:
            styles = yaml.load(stream)
            if styles.get('rules')==None:
                prib=print('Файл стилей должен содержать rules')
                return None
            elif styles.get('exceptions')==None:
                prib=print('Файл стилей должен содержать exceptions')
                return None
            return styles
        except yaml.YAMLError as exc:
            print(exc)

def GetLevelStyle(styles, admin_level):
    levelStyle = copy.copy(DefaultStyle)
    
    if styles['rules'].get(admin_level) != None:
        levelStyle = styles['rules'][admin_level]
    
    if levelStyle.get('color')==None:
        levelStyle['color'] = DefaultStyle['color']
    if levelStyle.get('width')==None:
        levelStyle['width'] = DefaultStyle['width']
    return levelStyle

def GetCurrentStyle(styles, levelStyle, feature):
    currentStyle = copy.copy(levelStyle)
    
    exception = styles['exceptions'].get(feature['properties']['name'])
    
    if exception != None:
        if exception.get('color')!=None:
            currentStyle['color'] = exception['color']
        if exception.get('width')!=None:
            currentStyle['width'] = exception['width']
    return currentStyle

def ToConvertCoordinate(c):
    if c[0]<0:
        c[0]=c[0]+360
    c[0]-=30

    c[0],c[1] = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), c[0], c[1])
    c[0]=90+c[0]/200000+15
    c[1]=90-c[1]/200000
    
    return c



#Переводит координаты в формат для отрисовки
def ToNormalizeCoordinates(p, p0, size):
    x1 = (p[0]-p0[0]) * size
    y1 = (p[1]-p0[1]) * size
    return (x1, y1)

def ToCheckPointVisibility(p, p0, p1):
    if p[0] > p0[0] and p[1] > p0[1] and p[0] < p1[0] and p[1] < p1[1]:
        return True
    else:
        return False
def ToCheckPolygonVisibility(minP, maxP, p0, p1):
    if maxP[0] < p0[0] or maxP[1] < p0[1] or minP[0] > p1[0] or minP[1] > p1[1]:
        return False
    else:
        return True
    
def ToDrawPolygon(draw, coordinates, size, p0, p1, colour="red", width=1):
    previousP = -1
    norm_P0 = ToNormalizeCoordinates(copy.copy(p0), p0, size)
    norm_P1 = ToNormalizeCoordinates(copy.copy(p1), p0, size)
    points=[]
    
    for p in coordinates:
        points.append(ToNormalizeCoordinates(p, p0, size))
        continue
        #Не используемый код, предназначенный для рисования линиями
        if previousP == -1:
            previousP = ToNormalizeCoordinates(p, p0, size)
        else:
            currentP = ToNormalizeCoordinates(p, p0, size)
            
            if ToCheckPointVisibility(previousP, norm_P0, norm_P1) or ToCheckPointVisibility(currentP, p0, p1):
                draw.line((previousP[0], previousP[1], currentP[0], currentP[1]), fill=colour, width=int(width*size/100))
            previousP = currentP
        #
    draw.polygon(points,outline = colour)


def ToDrawPicture(W, p0, p1, activeLevelDictionary, styles):
    H = W * (p1[1] - p0[1]) / (p1[0] - p0[0])
    size = W / (p1[0] - p0[0])
    
    image = Image.new("RGBA", (int(W),int(H)), (250,250,255,255))
    draw = ImageDraw.Draw(image)
    
    for i in reversed(range(len(data))):
        dataLevel = data[i]
        if activeLevelDictionary.get(i)!=None and activeLevelDictionary[i]:
            currentStyle = GetLevelStyle(styles, i)
            
            for feature in dataLevel:
                
                if not ToCheckPolygonVisibility(feature['properties']['minP'], feature['properties']['maxP'], p0, p1):
                    continue
                
                for a in feature['geometry']['coordinates']:
                    for b in a:
                        b = simplify_coords(b, 0.1/size)
                        ToDrawPolygon(draw, b, size, p0, p1, currentStyle['color'], currentStyle['width'])
    del(draw)
    
    return image

In [95]:
data = []
for i in range(10):
    data.append([])
data[2] = LoadData("admin_level_2.geojson")
data[3] = LoadData("admin_level_3.geojson")
data[6] = LoadData("admin_level_6.geojson")

In [63]:
styles = LoadStyles()

In [172]:
%timeit image = ToDrawPicture(1000, (115, 55), (120, 60), {2: False, 3: True, 6: True}, styles) #%timeit 
image = ToDrawPicture(1000, (115, 55), (120, 60), {2: True, 3: True, 6: True}, styles)

image.show()
#image.save("test.png", "PNG")

62.6 ms ± 689 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [152]:
a=[]
for i in range(1000000):
    a.append([0])

def test(a):
    for i in range(len(a)):
        a[i][0]=1
%timeit test(a)

44.2 ms ± 506 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [154]:
a=[]
for i in range(1000000):
    a.append([0])

def test(a):
    a=copy.copy(a)
    for i in a:
        i[0]=1
%timeit test(a)

40.4 ms ± 441 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
dataLevel_3[0]['properties']['admin_level']

'3'

In [147]:
data[2][0]['geometry']['coordinates'][28][0][0]

[188.4896180949552, 36.28147621122199]